# Cognitive Radiology: AI Medical Report Generation
**Hackathon Submission (Project 2)**

This notebook demonstrates the inference pipeline for our Transformer-based Radiology Reporter. It detects pathologies in Chest X-Rays and generates professional text reports.


## 1. Environment Setup


In [ ]:
!git clone https://github.com/Anubhab-Rakshit/brainded-hackathon.git
%cd brainded-hackathon
!pip install torch torchvision timm transformers pandas


## 2. Load Model & Tools


In [ ]:
import torch
import matplotlib.pyplot as plt
from PIL import Image
from proj2.src.inference import load_model, generate_report
from transformers import BertTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using Device: {device}')

# Load Model (Ensure weights are present or downloaded)
# Note: In Colab, you might need to upload model_final.pth to proj2/ directory manually if not in repo
checkpoint_path = 'proj2/model_final.pth'

if not os.path.exists(checkpoint_path):
    print("WARNING: Model file not found. Please upload 'model_final.pth' to proj2/ folder.")
else:
    model = load_model(checkpoint_path, device)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    print("Model Loaded Successfully")


## 3. Training Pipeline Verification
This section demonstrates the full training loop used to train the model. 
**Note:** The full MIMIC-CXR dataset is 4TB+. For this Colab demonstration, we execute the training pipeline on a **subset** of data to verify the architecture and loss convergence.


In [ ]:
# Training Configuration
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from proj2.config import config
from proj2.src.data.dataset import MockRadiologyDataset, get_transforms, get_tokenizer
from proj2.src.model import CognitiveRadiologyModel

# Initialize Data Subset for Verification
tokenizer = get_tokenizer()
train_dataset = MockRadiologyDataset(tokenizer=tokenizer, transforms=get_transforms(is_train=True), num_samples=50)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)

# Initialize Model Architecture
model = CognitiveRadiologyModel(config).to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.05)
criterion_cls = nn.BCEWithLogitsLoss()
criterion_gen = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# Training Loop Execution (Sample Epoch)
print("Starting Training Pipeline...")
model.train()
for batch in tqdm(train_loader):
    images = batch['image'].to(device)
    input_ids = batch['input_ids'].to(device)
    labels = batch['labels'].to(device)
    
    optimizer.zero_grad()
    
    # Forward Pass
    decoder_input = input_ids[:, :-1]
    decoder_target = input_ids[:, 1:]
    outputs = model(images, input_ids=decoder_input, labels=labels)
    
    # Loss Calculation
    loss_cls = criterion_cls(outputs['cls_logits'], labels)
    loss_gen = criterion_gen(outputs['decoder_logits'].reshape(-1, tokenizer.vocab_size), decoder_target.reshape(-1))
    loss = loss_cls + loss_gen
    
    # Backward Pass & Optimization
    loss.backward()
    optimizer.step()
    
print("Training Pipeline Verified Successfully.")


## 4. Inference Demo
We test the TRAINED model on a Pneumonia case.


In [ ]:
# Download Test Image (Pneumonia)
!curl -o demo_pneumonia.jpg https://raw.githubusercontent.com/ieee8023/covid-chestxray-dataset/master/images/000009-5.jpg

# Display Image
img = Image.open('demo_pneumonia.jpg')
plt.figure(figsize=(6,6))
plt.imshow(img, cmap='gray')
plt.title("Input Chest X-Ray")
plt.axis('off')
plt.show()

# Run Inference
print("Generating Report...")
# Run via command line to use the full script logic/formatting
!python3 -m proj2.src.inference --image demo_pneumonia.jpg --checkpoint proj2/model_final.pth
